In [ ]:
import numpy as np
import pandas as pd
import pandas_datareader.data as web
# length of time series
start, end = "1900-01-01", "2025-12-31"

# FRED ID
fred_ids = {
    "CPI": "CPIAUCSL",
    "PPI": "PPIACO",
    #"Unemployment": "UNRATE",
    #"IndustrialProduction": "INDPRO",
    #"M2": "M2SL",
    #"UST10Y": "GS10",
    #"UST3M": "GS3M",
    #"SP500": "SP500",
   
}

# obtain data
df = pd.DataFrame()
for name, fid in fred_ids.items():
    try:
        df[name] = web.DataReader(fid, "fred", start, end)
    except Exception as e:
        print(f"Error fetching {name} ({fid}): {e}")

# transfer to monthly frequency
df_monthly = df.resample("M").last()

# first order difference
returns = pd.DataFrame()
for col in df_monthly.columns:
    series = df_monthly[col].dropna()
    if (series <= 0).any():
        r = series.diff().dropna()
    else:
        r = np.log(series).diff().dropna()
    returns[col] = r

# skewness and excess kurtosis
stats = pd.DataFrame({
    "Skewness": returns.skew(),
    "Kurtosis": returns.kurtosis()
})

print(stats)

In [8]:
import re
import pandas as pd
import numpy as np
from typing import Literal, Optional

def load_uk_rpi_mixed_csv(
    path: str,
    value_col: int = 1,
    date_col: int = 0,
    keep_annual: Literal["drop", "jan", "dec"] = "drop",
    uppercase_month=True,
) -> pd.Series:
   
    raw = pd.read_csv(path, header=None, usecols=[date_col, value_col], dtype=str)
    raw = raw.rename(columns={date_col: "period", value_col: "value"})

    raw["period"] = raw["period"].astype(str).str.strip()
    raw["value"] = raw["value"].astype(str).str.strip()
    raw = raw.replace({"": np.nan, "nan": np.nan}).dropna(subset=["period", "value"])

    pat_year = re.compile(r"^\s*(\d{4})\s*$")
    pat_month = re.compile(r"^\s*(\d{4})\s+([A-Za-z]{3})\s*$")

    records = []

    for _, row in raw.iterrows():
        p = row["period"]
        v = row["value"]
        try:
            val = float(v)
        except:
            continue

        m_month = pat_month.match(p)
        m_year = pat_year.match(p)

        if m_month:
            yyyy = int(m_month.group(1))
            mon_abbr = m_month.group(2)
            if uppercase_month:
                mon_abbr = mon_abbr.upper()

            m_map = dict(JAN=1, FEB=2, MAR=3, APR=4, MAY=5, JUN=6,
                         JUL=7, AUG=8, SEP=9, OCT=10, NOV=11, DEC=12)
            if mon_abbr not in m_map:
                if mon_abbr.startswith("SE"):
                    mon_abbr = "SEP"
                if mon_abbr not in m_map:
                    continue
            mm = m_map[mon_abbr]
            dt = pd.Timestamp(year=yyyy, month=mm, day=1)  # 月初
            records.append((dt, val, "M"))

        elif m_year:
            if keep_annual == "drop":
                continue
            yyyy = int(m_year.group(1))
            mm = 1 if keep_annual == "jan" else 12
            dt = pd.Timestamp(year=yyyy, month=mm, day=1)
            records.append((dt, val, "Y"))

        else:
            continue

    if not records:
        raise ValueError("NO valid data.")

    df = pd.DataFrame(records, columns=["date", "value", "freq"]).sort_values("date")
    s = df.set_index("date")["value"].astype(float)

    s = s.asfreq("MS") 
    s.name = "UK_RPI"
    return s


In [ ]:
# RPI, obtaind fron ONS
uk_rpi = load_uk_rpi_mixed_csv("/Users/hanshuting/Desktop/RPI.csv", keep_annual="drop")

def to_monthly_growth(series: pd.Series) -> pd.Series:
    s = series.dropna()
    if (s <= 0).any():
        return s.diff().dropna()
    return np.log(s).diff().dropna()

rpi_growth = to_monthly_growth(uk_rpi)
skew_excess = rpi_growth.skew()           
kurt_excess = rpi_growth.kurtosis()       

print({"skew": skew_excess, "kurt_excess": kurt_excess})


In [14]:
import numpy as np
import pandas as pd
from dataclasses import dataclass
from typing import Dict, Any, Tuple
from scipy.optimize import minimize
from scipy.special import gammaln

EPS = 1e-12

# ---------- helpers: link functions ----------
def softplus(x):  # >0
    return np.log1p(np.exp(-np.abs(x))) + np.maximum(x, 0.0)

def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

# ---------- map raw -> constrained (omega>0, alpha>=0, beta>=0, alpha+beta<1) ----------
def map_garch_params(theta_raw: np.ndarray) -> Tuple[float, float, float]:
    # (omega, alpha, beta)
    log_omega, s_raw, r_raw = theta_raw[:3]
    omega = np.exp(log_omega) + EPS
    A = softplus(s_raw)
    B = softplus(r_raw)
    total = 1.0 + A + B
    alpha = A / total * (1.0 - 1e-3)
    beta  = B / total * (1.0 - 1e-3)
    return omega, alpha, beta

def logpdf_gauss(z: np.ndarray) -> np.ndarray:
    return -0.5 * (np.log(2.0 * np.pi) + z**2)

def logpdf_t_unitvar(z: np.ndarray, nu: float) -> np.ndarray:
    # z = sqrt((nu-2)/nu) * u, u ~ StudentT(nu)
    # => log f_Z(z) = log f_U(u) + log(du/dz) = log t_pdf(u;nu) + 0.5*log(nu/(nu-2))
    u = z * np.sqrt(nu / (nu - 2.0))
    c = 0.5 * np.log(nu / (nu - 2.0))
    # log c_nu = lgamma((nu+1)/2) - lgamma(nu/2) - 0.5*log(nu*pi)
    log_c = gammaln(0.5 * (nu + 1.0)) - gammaln(0.5 * nu) - 0.5 * (np.log(nu) + np.log(np.pi))
    return log_c - 0.5 * (nu + 1.0) * np.log1p((u**2) / nu) + c

def logpdf_beta_unitvar(z: np.ndarray, a: float, b: float) -> np.ndarray:
    # 若 Y ~ Beta(a,b) on (0,1), 则 Z = (Y - m)/s, m=a/(a+b), s=sqrt(ab/((a+b)^2 (a+b+1)))
    m = a / (a + b)
    s = np.sqrt(a * b / (((a + b) ** 2) * (a + b + 1.0)))
    y = s * z + m
    mask = (y > 0.0) & (y < 1.0)
    out = np.full_like(z, fill_value=-1e12, dtype=float)
    # log Beta(a,b) = lgamma(a)+lgamma(b)-lgamma(a+b)
    log_B = gammaln(a) + gammaln(b) - gammaln(a + b)
    # log f_Y(y) = (a-1)log y + (b-1)log(1-y) - log_B
    yy = np.clip(y[mask], EPS, 1.0 - EPS)
    log_fy = (a - 1.0) * np.log(yy) + (b - 1.0) * np.log1p(-yy) - log_B
    # f_Z(z) = f_Y(y) * |dy/dz| = f_Y(y) * s  ⇒ log f_Z = log f_Y + log s
    out[mask] = log_fy + np.log(s + EPS)
    return out

# ---------- GARCH(1,1) recursion ----------
def garch_sigma2(resid: np.ndarray, omega: float, alpha: float, beta: float) -> np.ndarray:
    n = resid.size
    sigma2 = np.empty(n, dtype=float)
    longrun = omega / max(1.0 - alpha - beta, 1e-6)
    sigma2[0] = longrun
    for t in range(1, n):
        sigma2[t] = omega + alpha * (resid[t-1] ** 2) + beta * sigma2[t-1]
        
        if sigma2[t] < 1e-12:
            sigma2[t] = 1e-12
    return sigma2


def nll_garch(theta_raw: np.ndarray, y: np.ndarray, dist: str) -> float:
   
    #   gaussian: [mu_raw, log_omega, s_raw, r_raw]
    #   t       : [mu_raw, log_omega, s_raw, r_raw, nu_raw]
    #   beta    : [mu_raw, log_omega, s_raw, r_raw, a_raw, b_raw]
    mu = theta_raw[0]
    omega, alpha, beta = map_garch_params(theta_raw[1:4])
    e = y - mu
    sigma2 = garch_sigma2(e, omega, alpha, beta)
    z = e / np.sqrt(sigma2)

    if dist == "gaussian":
        logfz = logpdf_gauss(z)
    elif dist == "t":
        nu = 4.0 + softplus(theta_raw[4])      # v>4 
        logfz = logpdf_t_unitvar(z, nu)
    elif dist == "beta":
        a = 2.0 + softplus(theta_raw[4])       # a,b >2
        b = 2.0 + softplus(theta_raw[5])
        logfz = logpdf_beta_unitvar(z, a, b)
    else:
        raise ValueError("Unknown dist")

    # p(e_t) = (1/sigma_t) * f_Z(z_t) ⇒ loglik = sum(log f_Z(z_t) - 0.5*log sigma2_t)
    ll = np.sum(logfz - 0.5 * np.log(sigma2))
    return float(-ll)

def information_criteria(n: int, k: int, loglik: float):
    aic  = 2*k - 2*loglik
    bic  = np.log(n)*k - 2*loglik
    if n > k + 1:
        aicc = aic + (2*k*(k+1)) / (n - k - 1)
    else:
        aicc = np.nan
    hqic = 2*k*np.log(np.log(max(n,3))) - 2*loglik
    return {"AIC": aic, "BIC": bic, "AICc": aicc, "HQIC": hqic}

@dataclass
class FitResult:
    dist: str
    params: Dict[str, float]
    loglik: float
    aic: float
    bic: float
    success: bool
    message: str
    ic: Dict[str, float] = None

def fit_garch_mle(y: np.ndarray, dist: str = "gaussian") -> FitResult:
    y = np.asarray(y, dtype=float)
    y = y[np.isfinite(y)]
    n = y.size

    if dist == "gaussian":
        x0 = np.array([np.mean(y), np.log(np.var(y)+1e-3), 0.1, 0.9])
    elif dist == "t":
        x0 = np.array([np.mean(y), np.log(np.var(y)+1e-3), 0.1, 0.9, np.log(np.e)])  # nu_raw≈1→nu≈4+1=5
    elif dist == "beta":
        x0 = np.array([np.mean(y), np.log(np.var(y)+1e-3), 0.1, 0.9, np.log(np.e), np.log(np.e)])
    else:
        raise ValueError("dist must be 'gaussian', 't', or 'beta'.")

    obj = lambda th: nll_garch(th, y, dist)
    res = minimize(obj, x0, method="L-BFGS-B")

    mu = res.x[0]
    omega, alpha, beta = map_garch_params(res.x[1:4])

    params = {"mu": mu, "omega": omega, "alpha": alpha, "beta": beta}
    k = 4
    if dist == "t":
        nu = 4.0 + softplus(res.x[4])
        params["nu"] = nu
        k += 1
    if dist == "beta":
        a = 2.0 + softplus(res.x[4])
        b = 2.0 + softplus(res.x[5])
        params["a"] = a
        params["b"] = b
        k += 2

    loglik = -obj(res.x)
    aic = 2 * k - 2 * loglik
    bic = np.log(n) * k - 2 * loglik
    ic = information_criteria(n, k, loglik)
    return FitResult(dist=dist, params=params, loglik=loglik,
                 aic=aic, bic=bic, success=bool(res.success),
                 message=res.message, ic=ic)

def to_returns_monthly(x: pd.Series) -> pd.Series:
    x = x.dropna()
    if (x <= 0).any():
        r = x.diff()
    else:
        r = np.log(x).diff()
    return r.dropna()

def compare_three(y: pd.Series) -> pd.DataFrame:
    out = []
    for d in ["gaussian", "t", "beta"]:
        fr = fit_garch_mle(y.values, dist=d)
        row = {"dist": d, "loglik": fr.loglik, "AIC": fr.aic, "BIC": fr.bic, "success": fr.success}
        row.update(fr.params)
        out.append(row)
    return pd.DataFrame(out).set_index("dist").sort_values("BIC")


In [ ]:
y1 = to_returns_monthly(df_monthly["CPI"])     
summary1 = compare_three(y1)
print(summary1)
fr_g1 = fit_garch_mle(y1.values, dist="gaussian")
fr_t1 = fit_garch_mle(y1.values, dist="t")
fr_b1 = fit_garch_mle(y1.values, dist="beta")

def ic_table(*fit_results):
    rows = []
    for fr in fit_results:
        rows.append({
            "dist": fr.dist.title(),
            "AIC": fr.ic["AIC"], "BIC": fr.ic["BIC"],
            "AICc": fr.ic["AICc"], "HQIC": fr.ic["HQIC"]
        })
    return pd.DataFrame(rows).set_index("dist").round(4)

df_ic1 = ic_table(fr_g1, fr_t1, fr_b1)
display(df_ic1)

In [ ]:
y2 = to_returns_monthly(df_monthly["PPI"])     
summary2 = compare_three(y2)
print(summary2)
fr_g2 = fit_garch_mle(y2.values, dist="gaussian")
fr_t2 = fit_garch_mle(y2.values, dist="t")
fr_b2 = fit_garch_mle(y2.values, dist="beta")

def ic_table(*fit_results):
    rows = []
    for fr in fit_results:
        rows.append({
            "dist": fr.dist.title(),
            "AIC": fr.ic["AIC"], "BIC": fr.ic["BIC"],
            "AICc": fr.ic["AICc"], "HQIC": fr.ic["HQIC"]
        })
    return pd.DataFrame(rows).set_index("dist").round(4)

df_ic2 = ic_table(fr_g2, fr_t2, fr_b2)
display(df_ic2)

In [149]:
from scipy.stats import norm, t as tdist, beta as betadist

def q_gauss(alpha): return float(norm.ppf(alpha))

def q_t_unitvar(alpha, nu):
    return float(np.sqrt((nu-2.0)/nu) * tdist.ppf(alpha, df=nu))

def q_beta_std(alpha, a, b):
    m = a/(a+b)
    s = np.sqrt(a*b/(((a+b)**2)*(a+b+1.0)))
    y = float(betadist.ppf(alpha, a, b))
    return (y - m)/s

def rvs_unitvar(dist: str, size: int, **kw):
    if dist == "gaussian":
        return np.random.randn(size)
    if dist == "t":
        nu = kw["nu"]; return np.sqrt((nu-2.0)/nu) * tdist.rvs(df=nu, size=size)
    if dist == "beta":
        a, b = kw["a"], kw["b"]
        y = betadist.rvs(a, b, size=size)
        m = a/(a+b); s = np.sqrt(a*b/(((a+b)**2)*(a+b+1.0)))
        return (y - m)/s
    raise ValueError("unknown dist")

# --- σ²_{T+1|T: T+h|T} ---
def forecast_sigma2_path(last_e2, last_sigma2, omega, alpha, beta, h):
    out = np.empty(h, dtype=float)
    s = omega + alpha*last_e2 + beta*last_sigma2
    out[0] = max(s, 1e-12)
    for i in range(1, h):
        s = omega + (alpha + beta) * out[i-1]
        out[i] = max(s, 1e-12)
    return out

# --- 1-step VaR ---
def var_1step(mu, sigma2_1, dist, **kw):
    s = np.sqrt(sigma2_1)
    alpha = kw.get("alpha", 0.01)
    if dist == "gaussian":
        q = q_gauss(alpha)
    elif dist == "t":
        q = q_t_unitvar(alpha, kw["nu"])
    elif dist == "beta":
        q = q_beta_std(alpha, kw["a"], kw["b"])
    else:
        raise ValueError("unknown dist")
    return mu + q * s

# --- h-step VaR（analytic/MC） ---
def var_hstep(mu, sigma2_path, dist, alpha, method="analytic", mc_paths=20000, **kw):
    h = len(sigma2_path)
    if method == "analytic":
        s_tot = np.sqrt(np.sum(sigma2_path))
        q = q_gauss(alpha)  
        return h*mu + q*s_tot
    elif method == "mc":
        Z = np.empty((mc_paths, h))
        if dist == "gaussian":
            for s in range(h): Z[:, s] = rvs_unitvar("gaussian", mc_paths)
        elif dist == "t":
            for s in range(h): Z[:, s] = rvs_unitvar("t", mc_paths, nu=kw["nu"])
        else:
            for s in range(h): Z[:, s] = rvs_unitvar("beta", mc_paths, a=kw["a"], b=kw["b"])
        R = (mu + Z * np.sqrt(sigma2_path)[None, :]).sum(axis=1)
        return float(np.quantile(R, alpha))
    else:
        raise ValueError("method must be 'analytic' or 'mc'")

# --- 1-step VaR（rolling）---
def one_step_var_series(y, fr, alpha):
    mu = fr.params["mu"]; omega = fr.params["omega"]
    a = fr.params["alpha"]; b = fr.params["beta"]
    y = np.asarray(y); e = y - mu
    sigma2 = garch_sigma2(e, omega, a, b)
    var = np.empty_like(y, dtype=float)
    if fr.dist == "gaussian":
        for t in range(y.size):
            var[t] = var_1step(mu, sigma2[t], "gaussian", alpha=alpha)
    elif fr.dist == "t":
        nu = fr.params["nu"]
        for t in range(y.size):
            var[t] = var_1step(mu, sigma2[t], "t", alpha=alpha, nu=nu)
    else:
        A, B = fr.params["a"], fr.params["b"]
        for t in range(y.size):
            var[t] = var_1step(mu, sigma2[t], "beta", alpha=alpha, a=A, b=B)
    return var

# --- h-step VaR（rolling）---
def h_step_var_series(y, fr, alpha, h, method="analytic", mc_paths=30000):
    mu = fr.params["mu"]; omega = fr.params["omega"]
    a = fr.params["alpha"]; b = fr.params["beta"]
    y = np.asarray(y); e = y - mu
    sigma2 = garch_sigma2(e, omega, a, b)
    var = np.empty_like(y, dtype=float)
    for t in range(y.size):
        last_e2 = e[t-1]**2 if t-1 >= 0 else omega/(1.0 - a - b)
        last_s2 = sigma2[t-1] if t-1 >= 0 else omega/(1.0 - a - b)
        s_path = forecast_sigma2_path(last_e2, last_s2, omega, a, b, h)
        if fr.dist == "gaussian":
            var[t] = var_hstep(mu, s_path, "gaussian", alpha, method, mc_paths)
        elif fr.dist == "t":
            var[t] = var_hstep(mu, s_path, "t", alpha, method, mc_paths, nu=fr.params["nu"])
        else:
            var[t] = var_hstep(mu, s_path, "beta", alpha, method, mc_paths,
                               a=fr.params["a"], b=fr.params["b"])
    return var

# --- VaR backtesting（UC/IND/CC） ---
from scipy.stats import chi2

def var_backtests(y, var, alpha):
    y = np.asarray(y); var = np.asarray(var)
    m = np.isfinite(y) & np.isfinite(var)
    y = y[m]; var = var[m]; n = y.size
    I = (y < var).astype(int)  

    # UC
    n1 = I.sum(); p_hat = n1 / n if n>0 else 0.0
    def _logp(p): 
        p = np.clip(p, 1e-12, 1-1e-12)
        return np.log(p)
    def L_bin(k, N, p):
        p = np.clip(p, 1e-12, 1-1e-12)
        return k*_logp(p) + (N-k)*_logp(1-p)
    LR_uc = -2*(L_bin(n1, n, alpha) - L_bin(n1, n, p_hat))
    p_uc  = 1 - chi2.cdf(LR_uc, 1)

    # IND
    I0, I1 = I[:-1], I[1:]
    N00 = np.sum((I0==0)&(I1==0))
    N01 = np.sum((I0==0)&(I1==1))
    N10 = np.sum((I0==1)&(I1==0))
    N11 = np.sum((I0==1)&(I1==1))
   
    pi01 = np.clip(N01 / max(N00+N01,1), 1e-12, 1-1e-12)
    pi11 = np.clip(N11 / max(N10+N11,1), 1e-12, 1-1e-12)
    def L_markov(N00,N01,N10,N11,pi01,pi11):
        s = 0.0
        for Nxy, p in [(N01,pi01),(N00,1-pi01),(N11,pi11),(N10,1-pi11)]:
            if p in (0,1): return -np.inf
            s += Nxy*np.log(p)
        return s
    L1 = L_markov(N00,N01,N10,N11,pi01,pi11)
    T = N00+N01+N10+N11
    pbar = np.clip((N01+N11) / max(N00+N01+N10+N11,1), 1e-12, 1-1e-12)
    L0 = L_markov(N00,N01,N10,N11,pbar,pbar)
    LR_ind = -2*(L0 - L1); p_ind = 1 - chi2.cdf(LR_ind, 1)

    # CC
    LR_cc = LR_uc + LR_ind; p_cc = 1 - chi2.cdf(LR_cc, 2)

    return {"n": int(n), "violations": int(n1), "hit_rate": p_hat,
            "LR_uc": float(LR_uc), "p_uc": float(p_uc),
            "LR_ind": float(LR_ind), "p_ind": float(p_ind),
            "LR_cc": float(LR_cc), "p_cc": float(p_cc)}


In [ ]:
y1 = to_returns_monthly(df_monthly["PPI"]).values  
fr_g = fit_garch_mle(y1, dist="gaussian")
fr_t = fit_garch_mle(y1, dist="t")
fr_b = fit_garch_mle(y1, dist="beta")
alpha = 0.01
var1_t = one_step_var_series(y1, fr_t, alpha)
bt1_t  = var_backtests(y1, var1_t, alpha)
h = 5
varh_t = h_step_var_series(y1, fr_t, alpha, h, method="mc", mc_paths=30000)

def rolling_sum(y: np.ndarray, h: int) -> np.ndarray:
    y = np.asarray(y, float)
    n = y.size
    if h == 1:
        return y.copy()
    out = np.full(n, np.nan)
    conv = np.convolve(y, np.ones(h), mode="valid")
    out[h-1:] = conv
    return out

Rh   = rolling_sum(y1, h)
bt_h = var_backtests(Rh, varh_t, alpha)

results = {
    "Gaussian": fr_g.ic,
    "t": fr_t.ic,
    "Beta": fr_b.ic
}
df_ic = pd.DataFrame(results).T   
print(df_ic)


df_bt = pd.DataFrame({
    "t 1-step": bt1_t,
    "t h-step": bt_h
}).T
print(df_bt)



In [ ]:
alpha = 0.05
h = 3 
y1 = to_returns_monthly(df_monthly["CPI"]).values  
var11 = {
    "Gaussian": one_step_var_series(y1, fr_g, alpha),
    "t":        one_step_var_series(y1, fr_t, alpha),
    "Beta":     one_step_var_series(y1, fr_b, alpha),
}
bt11 = {name: var_backtests(y1, v, alpha) for name, v in var11.items()}

Rh1 = rolling_sum(y1, h)
varh1 = {
    "Gaussian": h_step_var_series(y1, fr_g1, alpha, h, method="mc", mc_paths=30000),
    "t":        h_step_var_series(y1, fr_t1, alpha, h, method="mc", mc_paths=30000),
    "Beta":     h_step_var_series(y1, fr_b1, alpha, h, method="mc", mc_paths=30000),
}
bth1 = {name: var_backtests(Rh, v, alpha) for name, v in varh1.items()}
df_bt11 = pd.DataFrame(bt11).T.loc[:, ["n","violations","hit_rate","LR_uc","p_uc","LR_ind","p_ind","LR_cc","p_cc"]]
df_bth1 = pd.DataFrame(bth1).T.loc[:, ["n","violations","hit_rate","LR_uc","p_uc","LR_ind","p_ind","LR_cc","p_cc"]]

df_all1 = pd.concat(
    {
        "1-step": df_bt11,
        f"{h}-step": df_bth1
    },
    axis=0
)
df_all1 = df_all1.round(4)

from IPython.display import display
display(df_all1)

In [ ]:
var12 = {
    "Gaussian": one_step_var_series(y2, fr_g, alpha),
    "t":        one_step_var_series(y2, fr_t, alpha),
    "Beta":     one_step_var_series(y2, fr_b, alpha),
}
bt12 = {name: var_backtests(y2, v, alpha) for name, v in var12.items()}

Rh2 = rolling_sum(y2, h)
varh2 = {
    "Gaussian": h_step_var_series(y2, fr_g2, alpha, h, method="mc", mc_paths=30000),
    "t":        h_step_var_series(y2, fr_t2, alpha, h, method="mc", mc_paths=30000),
    "Beta":     h_step_var_series(y2, fr_b2, alpha, h, method="mc", mc_paths=30000),
}
bth2 = {name: var_backtests(Rh, v, alpha) for name, v in varh2.items()}

df_bt12 = pd.DataFrame(bt12).T.loc[:, ["n","violations","hit_rate","LR_uc","p_uc","LR_ind","p_ind","LR_cc","p_cc"]]
df_bth2 = pd.DataFrame(bth2).T.loc[:, ["n","violations","hit_rate","LR_uc","p_uc","LR_ind","p_ind","LR_cc","p_cc"]]

df_all2 = pd.concat(
    {
        "1-step": df_bt12,
        f"{h}-step": df_bth2
    },
    axis=0
)
df_all2 = df_all2.round(4)

display(df_all2)


In [ ]:
rpi_growth = to_monthly_growth(uk_rpi)

fr_g8 = fit_garch_mle(rpi_growth.values, dist="gaussian")
fr_t8 = fit_garch_mle(rpi_growth.values, dist="t")
fr_b8 = fit_garch_mle(rpi_growth.values, dist="beta")

alpha = 0.01
var1_t = one_step_var_series(rpi_growth.values, fr_t, alpha)
bt1_t  = var_backtests(rpi_growth.values, var1_t, alpha)
print(bt1_t)

In [ ]:
def ic_table(*fit_results):
    rows = []
    for fr in fit_results:
        rows.append({
            "dist": fr.dist.title(),
            "AIC": fr.ic["AIC"], "BIC": fr.ic["BIC"],
            "AICc": fr.ic["AICc"], "HQIC": fr.ic["HQIC"]
        })
    return pd.DataFrame(rows).set_index("dist").round(4)

df_ic8 = ic_table(fr_g8, fr_t8, fr_b8)
print(df_ic8)
def backtest_tables(y, fr_g, fr_t, fr_b, alpha=0.01, h=3, method="mc", mc_paths=30000):
    # 1-step
    var1 = {
        "Gaussian": one_step_var_series(y, fr_g, alpha),
        "t":        one_step_var_series(y, fr_t, alpha),
        "Beta":     one_step_var_series(y, fr_b, alpha),
    }
    bt1 = {k: var_backtests(y, v, alpha) for k, v in var1.items()}
    df_bt1 = (pd.DataFrame(bt1).T
                .loc[:, ["n","violations","hit_rate","LR_uc","p_uc","LR_ind","p_ind","LR_cc","p_cc"]]
                .round(4))

    # h-step
    Rh = rolling_sum(y, h)
    varh = {
        "Gaussian": h_step_var_series(y, fr_g, alpha, h, method=method, mc_paths=mc_paths),
        "t":        h_step_var_series(y, fr_t, alpha, h, method=method, mc_paths=mc_paths),
        "Beta":     h_step_var_series(y, fr_b, alpha, h, method=method, mc_paths=mc_paths),
    }
    bth = {k: var_backtests(Rh, v, alpha) for k, v in varh.items()}
    df_bth = (pd.DataFrame(bth).T
                .loc[:, ["n","violations","hit_rate","LR_uc","p_uc","LR_ind","p_ind","LR_cc","p_cc"]]
                .round(4))
    return df_bt1, df_bth

df_bt18, df_bth8 = backtest_tables(rpi_growth.values, fr_g8, fr_t8, fr_b8, alpha=0.05, h=3)
#print("1-step VaR backtests:\n", df_bt1, "\n")
#print("3-step VaR backtests:\n", df_bth)
from IPython.display import display
#display(df_ic)
display(df_bt18)
display(df_bth8)


In [ ]:
for col in df.columns:
    print(col, df[col].first_valid_index(), "→", df[col].last_valid_index())


In [ ]:
import pandas as pd
from pandas_datareader import data as web

start, end = "1900-01-01", "2025-12-31"

FRED_IDS = {
    "Euro_HICP": "CP0000EZ19M086NEST",     
    "Japan_CPI": "JPNCPIALLMINMEI",        
    "China_CPI": "CHNCPIALLMINMEI",      
}

df_PI = pd.DataFrame()
for name, fid in FRED_IDS.items():
    try:
        s = web.DataReader(fid, "fred", start, end)
        s.name = name
        df_PI[name] = s
    except Exception as e:
        print(f"[WARN] failed: {name} ({fid}): {e}")

#print(df_PI.head())


In [ ]:
for col in df_PI.columns:
    print(col, df_PI[col].first_valid_index(), "→", df_PI[col].last_valid_index())


In [ ]:
import numpy as np

def to_monthly_index(df: pd.DataFrame) -> pd.DataFrame:
    return df.resample("M").last().asfreq("M")

def to_growth(series: pd.Series) -> pd.Series:
    s = series.dropna()
    if (s <= 0).any():
        return s.diff().dropna()
    else:
        return np.log(s).diff().dropna()

df_monthly = to_monthly_index(df_PI)
rets = df_monthly.apply(to_growth).dropna(how="all")

print("usable columns:", list(rets.columns))


In [ ]:
moments = pd.DataFrame({
    "n": rets.count(),
    "skew": rets.skew(),
    "kurt_excess": rets.kurtosis(),
})
print(moments.round(4))


In [ ]:
col = "Euro_HICP" 
if col not in rets.columns:
    raise ValueError(f"{col} not in the data, can use column:{list(rets.columns)}")

y = rets[col].dropna().values

fr_g = fit_garch_mle(y, dist="gaussian")
fr_t = fit_garch_mle(y, dist="t")
fr_b = fit_garch_mle(y, dist="beta")

df_ic = pd.DataFrame({
    "Gaussian": fr_g.ic,
    "t": fr_t.ic,
    "Beta": fr_b.ic
}).T
print(df_ic.round(4))

# VaR test（1-step / h-step）
from IPython.display import display

alpha = 0.05; h = 10
df_bt1, df_bth = backtest_tables(y, fr_g, fr_t, fr_b, alpha=alpha, h=h, method="mc", mc_paths=30000)
display(df_bt1)   # 1-step
display(df_bth)   # h-step


In [ ]:
col4 = "Euro_HICP"  
if col4 not in rets.columns:
    raise ValueError(f"{col4} not in the data, can use column:{list(rets.columns)}")

y4 = rets[col4].dropna().values

fr_g4 = fit_garch_mle(y4, dist="gaussian")
fr_t4 = fit_garch_mle(y4, dist="t")
fr_b4 = fit_garch_mle(y4, dist="beta")

df_ic4 = pd.DataFrame({
    "Gaussian": fr_g4.ic,
    "t": fr_t4.ic,
    "Beta": fr_b4.ic
}).T
print(df_ic4.round(4))

In [ ]:
alpha = 0.05; h = 3
df_bt14, df_bth4 = backtest_tables(y4, fr_g4, fr_t4, fr_b4, alpha=alpha, h=h, method="mc", mc_paths=30000)
display(df_bt1)   # 1-step
display(df_bth4)   # h-step

In [ ]:
col5 = "China_CPI"  
if col5 not in rets.columns:
    raise ValueError(f"{col5} not in the data, can use column:{list(rets.columns)}")

y5 = rets[col5].dropna().values

fr_g5 = fit_garch_mle(y5, dist="gaussian")
fr_t5 = fit_garch_mle(y5, dist="t")
fr_b5 = fit_garch_mle(y5, dist="beta")

df_ic5 = pd.DataFrame({
    "Gaussian": fr_g5.ic,
    "t": fr_t5.ic,
    "Beta": fr_b5.ic
}).T
print(df_ic5.round(4))

In [ ]:
alpha = 0.05; h = 3
df_bt15, df_bth5 = backtest_tables(y5, fr_g5, fr_t5, fr_b5, alpha=alpha, h=h, method="mc", mc_paths=30000)
display(df_bt15)   # 1-step
display(df_bth5)   # h-step


In [ ]:
col6 = "Japan_CPI"  
if col6 not in rets.columns:
    raise ValueError(f"{col6} not in the data, can use column:{list(rets.columns)}")

y6 = rets[col6].dropna().values

fr_g6 = fit_garch_mle(y6, dist="gaussian")
fr_t6 = fit_garch_mle(y6, dist="t")
fr_b6 = fit_garch_mle(y6, dist="beta")

df_ic6 = pd.DataFrame({
    "Gaussian": fr_g6.ic,
    "t": fr_t6.ic,
    "Beta": fr_b6.ic
}).T
print(df_ic6.round(4))

In [ ]:
alpha = 0.05; h = 3
df_bt16, df_bth6 = backtest_tables(y6, fr_g6, fr_t6, fr_b6, alpha=alpha, h=h, method="mc", mc_paths=30000)
display(df_bt16)   # 1-step
display(df_bth6)   # h-step


In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import norm
import numpy as np
import pandas as pd

# --- Put your series here ---
series_dict = {
    "US CPI": returns["CPI"].dropna(),
    "UK RPI": rpi_growth.dropna(),
    "Euro HICP": rets["Euro_HICP"].dropna(),
    "Japan CPI": rets["Japan_CPI"].dropna(),
    "China CPI": rets["China_CPI"].dropna(),
    "US PPI": returns["PPI"].dropna()
}

# convert to % growth
series_dict = {k: v * 100 for k, v in series_dict.items()}

# global axis limits
all_data = pd.concat(series_dict.values())
x_lo, x_hi = np.quantile(all_data, [0.005, 0.995])   # same x range
y_max = 0

# precompute global y max for density
for s in series_dict.values():
    mu, sd = s.mean(), s.std(ddof=0)
    x = np.linspace(x_lo, x_hi, 400)
    hist_vals, _ = np.histogram(s[(s >= x_lo) & (s <= x_hi)], bins=30, density=True)
    y_max = max(y_max, hist_vals.max(), norm.pdf(x, mu, sd).max())

# --- Plot each series separately but consistent scales ---
for name, s in series_dict.items():
    mu, sd = s.mean(), s.std(ddof=0)
    x = np.linspace(x_lo, x_hi, 400)

    plt.figure(figsize=(10,9))
    plt.hist(s[(s >= x_lo) & (s <= x_hi)], bins=30, density=True,
             edgecolor="black", alpha=0.7, color="steelblue")
    if sd > 0:
        plt.plot(x, norm.pdf(x, mu, sd), "r", lw=2, label="Fitted Normal")

    #plt.title(f"Skewness={s.skew():.3f}, Excess Kurtosis={s.kurtosis():.3f}", fontsize=20)
    plt.xlabel("Monthly log-difference", fontsize=30)
    plt.ylabel("Density", fontsize=30)
    plt.xlim(x_lo, x_hi)
    plt.ylim(0, y_max * 1.05)   # same y scale for all
    plt.legend(loc="upper right", fontsize=28) 
    plt.tick_params(axis='x', labelsize=30)  
    plt.tick_params(axis='y', labelsize=30)  
    plt.savefig(f"{name.replace(' ', '_')}_Thist.pdf", dpi=300)
    plt.tight_layout()
    plt.show()
